In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
from transformers import MT5ForConditionalGeneration, MT5Tokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from nltk.translate.bleu_score import corpus_bleu
from nltk.tokenize import word_tokenize
import nltk


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\io\image.py:13: UserWarning: Failed to load image Python extension: Could not find module 'C:\Users\ASUS\AppData\Local\Programs\Python\Python310\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [3]:

nltk.download('punkt')

# Load dataset
dset = load_dataset("SEACrowd/liputan6", trust_remote_code=True)

# Use subsets of the dataset
train_data = dset["train"].select(range(1000))
val_data = dset["validation"].select(range(200))
test_data = dset["test"].select(range(200))

# Load tokenizer and model
model_name = "microsoft/Phi-3.5-mini-instruct"
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ASUS\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'LlamaTokenizer'. 
The class this function is called from is 'T5Tokenizer'.


TypeError: not a string

In [3]:

# Preprocessing function
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["document"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["summary"], max_length=128, truncation=True, padding=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Tokenize dataset
tokenized_train = train_data.map(preprocess_function, batched=True, remove_columns=train_data.column_names)
tokenized_val = val_data.map(preprocess_function, batched=True, remove_columns=val_data.column_names)
tokenized_test = test_data.map(preprocess_function, batched=True, remove_columns=test_data.column_names)


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\tokenization_utils_base.py:3953: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map: 100%|██████████| 200/200 [00:00<00:00, 689.68 examples/s]


In [4]:

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
)

# Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)


c:\Users\ASUS\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
C:\Users\ASUS\AppData\Local\Temp\ipykernel_23504\1686153763.py:19: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [5]:

# Fine-tune the model
trainer.train()


  1%|▏         | 10/750 [01:32<1:49:56,  8.91s/it]

{'loss': 42.076, 'grad_norm': 5800.24951171875, 'learning_rate': 1.9733333333333336e-05, 'epoch': 0.04}


  3%|▎         | 20/750 [02:56<1:39:38,  8.19s/it]

{'loss': 41.7033, 'grad_norm': 4085.09521484375, 'learning_rate': 1.9466666666666668e-05, 'epoch': 0.08}


  4%|▍         | 30/750 [04:17<1:37:12,  8.10s/it]

...

                                                   
100%|██████████| 750/750 [3:10:07<00:00,  7.48s/it]

{'eval_loss': 12.500824928283691, 'eval_runtime': 84.9652, 'eval_samples_per_second': 2.354, 'eval_steps_per_second': 0.588, 'epoch': 3.0}


There were missing keys in the checkpoint model loaded: ['encoder.embed_tokens.weight', 'decoder.embed_tokens.weight'].
100%|██████████| 750/750 [3:10:16<00:00, 15.22s/it]

{'train_runtime': 11416.2267, 'train_samples_per_second': 0.263, 'train_steps_per_second': 0.066, 'train_loss': 26.477442667643228, 'epoch': 3.0}


TrainOutput(global_step=750, training_loss=26.477442667643228, metrics={'train_runtime': 11416.2267, 'train_samples_per_second': 0.263, 'train_steps_per_second': 0.066, 'total_flos': 1586249072640000.0, 'train_loss': 26.477442667643228, 'epoch': 3.0})

In [7]:
from nltk.translate.bleu_score import sentence_bleu

# Evaluate BLEU score
def compute_bleu(data):
    references = [[word_tokenize(summary)] for summary in data["summary"]]
    predictions = []
    for article in data["document"]:
        inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(word_tokenize(pred_summary))
    return corpus_bleu(references, predictions)

average_bleu = compute_bleu(test_data)
print(f"Average BLEU Score: {average_bleu}")

# Predict on test data and print BLEU score for each sample
for i in range(10):  # Predict and display summaries for first 10 examples
    article = test_data[i]["document"]
    reference_summary = test_data[i]["summary"]

    # Generate prediction
    inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tokenize references and prediction
    reference_tokens = word_tokenize(reference_summary)
    prediction_tokens = word_tokenize(pred_summary)

    # Compute BLEU score for the sample
    sample_bleu = sentence_bleu([reference_tokens], prediction_tokens)

    print(f"Article: {article}...")
    print(f"Reference Summary: {reference_summary}")
    print(f"Predicted Summary: {pred_summary}")
    print(f"BLEU Score: {sample_bleu}\n")


Average BLEU Score: 0.0004225577576027379
Article: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum pat

In [ ]:
from nltk.translate.bleu_score import sentence_bleu

# Evaluate BLEU score
def compute_bleu(data):
    references = [[word_tokenize(summary)] for summary in data["summary"]]
    predictions = []
    for article in data["document"]:
        inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
        outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
        pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
        predictions.append(word_tokenize(pred_summary))
    return corpus_bleu(references, predictions)

average_bleu = compute_bleu(test_data)
print(f"Average BLEU Score: {average_bleu}")

# Predict on test data and print BLEU score for each sample
for i in range(10):  # Predict and display summaries for first 10 examples
    article = test_data[i]["document"]
    reference_summary = test_data[i]["summary"]

    # Generate prediction
    inputs = tokenizer("summarize: " + article, return_tensors="pt", max_length=512, truncation=True)
    outputs = model.generate(**inputs, max_length=128, num_beams=4, early_stopping=True)
    pred_summary = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Tokenize references and prediction
    reference_tokens = word_tokenize(reference_summary)
    prediction_tokens = word_tokenize(pred_summary)

    # Compute BLEU score for the sample
    sample_bleu = sentence_bleu([reference_tokens], prediction_tokens)

    print(f"Article: {article}...")
    print(f"Reference Summary: {reference_summary}")
    print(f"Predicted Summary: {pred_summary}")
    print(f"BLEU Score: {sample_bleu}\n")


Average BLEU Score: 0.0004225577576027379
Article: Liputan6 . com , Bangka : Kapal patroli Angkatan Laut Republik Indonesia , Belinyu , baru-baru ini , menangkap tiga kapal nelayan berbendera Thailand , yakni KM Binatama , KM Sumber Jaya II , dan KM Mataram di Perairan Belitung Utara . Ketiga kapal itu ditangkap karena melanggar zona ekonomi ekslusif Indonesia . Saat ini , kapal-kapal itu diamankan di Pos Lanal Pelabuhan Pangkalan Balam , Bangka-Belitung . Menurut Komandan Pangkalan TNI AL Bangka Letnan Kolonel Laut Fredy Egam , selain menangkap tiga kapal , ALRI juga memeriksa 43 anak buah kapal . Mereka disergap saat sedang mengangkat jaring pukat harimau di Perairan Belitung Utara . Dari jumlah itu , hanya enam orang yang dijadikan tersangka , yakni tiga nahkoda dan tiga kepala kamar mesin kapal . Sedangkan ABK yang lain akan dideportasi ke negara asalnya . Meski berhasil menahan enam tersangka , TNI AL gagal mengamankan ikan tangkapan nelayan Thailand tersebut . Sebab , sebelum pat